In [1]:
#install DEAP
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


In [2]:
#import required packages

import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [3]:

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", partial(random.randint, -1, 1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


In [4]:
#def main
# results

def main():
    random.seed(2)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 50, stats=mstats,
                                   halloffame=None, verbose=True)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    main()

   	      	                        fitness                        	                     size                     
   	      	-------------------------------------------------------	----------------------------------------------
gen	nevals	avg    	gen	max    	min     	nevals	std    	avg    	gen	max	min	nevals	std   
0  	300   	2.01574	0  	21.8347	0.165572	300   	2.65547	3.54333	0  	7  	2  	300   	1.5016
1  	171   	1.08236	1  	16.1606	0.165572	171   	1.07706	3.60333	1  	11 	1  	171   	1.74528
2  	169   	0.943453	2  	18.9739	0.165572	169   	1.43192	3.80333	2  	10 	1  	169   	1.87741
3  	156   	0.722882	3  	4.09456	0.165572	156   	0.451121	3.75   	3  	12 	1  	156   	2.12466
4  	166   	0.723457	4  	6.50856	0.165572	166   	0.59796 	4.44333	4  	12 	1  	166   	2.4386 
5  	161   	0.635288	5  	4.09456	0.146271	161   	0.497803	5.25667	5  	12 	1  	161   	2.50947
6  	174   	0.712857	6  	13.8349	0.165572	174   	1.29506 	6.49   	6  	15 	1  	174   	2.49731
7  	170   	0.555053	7  	4.55733	0.080882	170 